In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
import yellowbrick
import pycaret


In [7]:
from library.common import Database, Core
from pathlib import Path
import pickle

db = Database()
db_table = 'owid_co2_greenhouse_gas_emissions'
owid_df = pd.read_sql(db_table, db.config)
print("Data loaded")

core = Core()
# Clean up data
# NULL in ISO CODE are regional data
# Trim the dataset to remove regional and world aggregate
owid_df = owid_df.dropna(subset = ['iso_code']).copy()
owid_df = owid_df[~owid_df['iso_code'].isin(core.excluded_features)].copy()

Data loaded


In [8]:
# get the end of the dataset

param_dir = Path.cwd().parent/r'data/processed'
full_path = param_dir/'params_forecast_params.pkl'
full_path = full_path.as_posix()
with open(full_path, 'rb') as pkl:
    last = pickle.load(pkl)
    last_year = list(last.values())[0]

In [9]:
# Process the data for analysis
dataset = owid_df[owid_df['year'].eq(last_year)]
dataset = dataset[core.base_features].copy()
iso_code = dataset['iso_code']
dataset = dataset[core.base_features[2:]]
dataset = dataset.fillna(0).reset_index(drop = True)

print('Dataset prepared')

Dataset prepared


In [10]:
from yellowbrick.classifier import ClassificationReport


def visualize_model(X, y, estimator, **kwargs):
    """
    Test various estimators.
    """
    y = LabelEncoder().fit_transform(y)
    model = Pipeline([
        ('one_hot_encoder', OneHotEncoder()),
        ('estimator', estimator)
    ])

    # Instantiate the classification model and visualizer
    visualizer = ClassificationReport(
        model, classes=['edible', 'poisonous'],
        cmap="YlGn", size=(600, 360), **kwargs
    )
    visualizer.fit(X, y)
    visualizer.score(X, y)
    visualizer.show()

for model in models:
    visualize_model(X, y, model)


NameError: name 'models' is not defined